Question

In [1]:
# Build a symmetric block cipher with the following requirements:
# Must be a Feistel structure with 16 rounds
# Input plaintext = 32-bit block
# Output ciphertext = 32-bit block
# Key = 32 bits, producing subkey of 16 bits for each round
# The function F must implements operations defined on GF(2^16)
# Implements CBC mode for plaintext longer than 32 bits

Extra Methods

In [2]:
def add_padding(text, block_size):
    temp = (len(text) % block_size)
    if(temp == 0):
       padding_length = temp
    else:
      padding_length = block_size - (len(text) % block_size)
    padded_text = text + '0' * padding_length
    return padded_text
#added padding to make sure the CBC function works

def binary_converter(string):
    result = ''

    for char in string:
        binary_char = bin(ord(char))[2:].zfill(8)  #convert character to ASCII code, then to binary
        result += binary_char

    return result

def string_converter(string):
    blocks = [string[i:i+8] for i in range(0, len(string), 8)] #split the binary string into 8-bit blocks

    letters = [chr(int(chunk, 2)) for chunk in blocks] #convert each block back to ASCII character

    return ''.join(letters)

def remove_padding(text):
    temp = text[-8:]

    while(temp == '00000000'):
      text = text[:-8]
      temp = text[-8:]

    return format(text)

Feistel Methods

In [3]:
def AdditionGF16(text, key):  #XOR
    result = text ^ key
    return (result)


def SubKeyGen(key, round): #Complex Subkey Generator
    Rkey = (key << round | key >> (32 - round)) & 0xFFFFFFFF #Permutating the key by shifting it based on the round number
    left = Rkey >> 16
    right = Rkey & 0xFFFF

    subkey = AdditionGF16(left, right) #XOR the right and left side to get the round key

    return (subkey)


def Feistel(text, key, round = 0):
  if(round<16):

    subkey = SubKeyGen(key, round)

    left = text >> 16
    right = text & 0xFFFF
    F = AdditionGF16(right, subkey) #F Function

    Nright = AdditionGF16(left, F)
    Nleft = right

    newtext = (Nleft << 16) | Nright
    print('Round ', round+1, ' result :' , format(newtext, '032b'))

    return Feistel(newtext, key, round + 1)

  else:
    left = text >> 16
    right = text & 0xFFFF
    final = (right << 16) | left # Last Swap for Output 17
    return (final)
    return block ^ key


CBC Methods

In [4]:
def CBC(text, key): #CBC Funcion
  round = int(len(text) / len(key))
  temp = round
  n = 0
  keys = int(key, 2)
  c = keys
  count = 1
  encrypted_blocks = []
  while(round>0):
    print('C : ', format(c, '032b'))
    temp = int(text[n:n+len(key)], 2)
    res = AdditionGF16(c, temp)
    result = Feistel(res, keys)
    c = result
    round -= 1
    n += len(key)
    count += 1
    encrypted_blocks.append(format(result, '032b'))


  encrypted_text = ''.join(encrypted_blocks)
  return encrypted_text

DECRYPT

In [5]:
def ReverseFeistel(text, key, round = 0):
    if(round<16):

      subkey = SubKeyGen(key, 15 - round)

      left = text >> 16
      right = text & 0xFFFF
      F = AdditionGF16(right, subkey) #F Function

      Nright = AdditionGF16(left, F)
      Nleft = right

      newtext = (Nleft << 16) | Nright
      print('Round ', round+1, ' result :' , format(newtext, '032b'))

      return ReverseFeistel(newtext, key, round + 1)

    else:
      left = text >> 16
      right = text & 0xFFFF
      final = (right << 16) | left # Last Swap for Output 17
      return (final)


def CBC_decrypt(text, key): #decryption of CBC function
    round = int(len(text) / len(key))
    key_int = int(key, 2)
    block_size = len(key)
    decrypted_text = ''
    prev_block = key_int  # Initialize previous block with key for CBC decryption
    n = 0
    while (round>0):
        block = int(text[n:n + block_size], 2)
        print('C : ', format(block, '032b'))
        decrypted_block = ReverseFeistel(block, key_int)
        plain_block = decrypted_block ^ prev_block
        prev_block = block  # Update previous block for next iteration
        decrypted_text += format(plain_block, '032b')
        n += block_size
        round -= 1
    return decrypted_text

HASH

In [6]:
def rotate(key):
    right = key >> 15
    left = key & 0b111111111111111
    shifted = left << 1 | right
    return(shifted)

def HashPad(M):
    l = len(M)
    while(l%16!=0):
      M = M + '0'
      l = len(M)

    return(M)

def HMAC(M,HashK):
    ipad = 0b0011011000110110 #36 H repeated twice for 16 block
    opad = 0b0101110001011100 #5C H repeated twice for 16 block

    tempK = int(HashK,2)
    Ki = tempK ^ ipad
    Ko = tempK ^ opad

    tempM = M
    IV = 0b0000000000000000 ^ Ki
    while(len(tempM)>0):
      IV = rotate(IV)
      temp = int(tempM[:16],2)
      IV = IV ^ temp
      tempM = tempM[16:]

    print('Hash               : ', format(IV,'016b'))

    Ko = rotate(Ko)
    IV = IV ^ Ko

    return(format(IV, '016b'))


CODE RUNNER

In [7]:
plain_text = 'Hai'
key  = '11100101101011100101101000001111'
hashkey = '1000'

print('PlainText          : ', plain_text)

text = binary_converter(plain_text)
print('BinaryText         : ', text)

HashPadded = HashPad(text)
print('\nHashPadded         : ', HashPadded)
H = HMAC(HashPadded, hashkey)
print('HMAC               : ', H)

#Add padding as necessary
text = add_padding(text, len(key))
print('\nPaddedText         : ', text)

print('\nENCRYPTION\n')
#encryption
if(len(text)>32):
  CipherText = CBC(text, key)
else:
  text = int(text, 2)
  key = int(key, 2)
  CipherText = format(Feistel(text, key),'032b')


print("\nCipherBinary     : ", CipherText)
print("\nFinal(with Hash) : ", H + CipherText)

#CipherText = string_converter(CipherText)

#print("CipherText       : ", CipherText,)

#CipherText = binary_converter(CipherText) #turn back the string back to binary for decryption

#print('\nDECRYPTION\n')

#if(len(CipherText)>32):
#  DecryptedText = CBC_decrypt(CipherText, key)
#  print('\nDecryptedText      : ', DecryptedText)
#else:
#  CipherText = int(CipherText, 2)
#  DecryptedText = format(ReverseFeistel(CipherText, key), '032b')
#  print('\nDecryptedText      : ', DecryptedText)

#stringbin = remove_padding(DecryptedText) #remove padding as necessary

#string = string_converter(stringbin)
#print('Decrypted Message  : ',string)

PlainText          :  Hai
BinaryText         :  010010000110000101101001

HashPadded         :  01001000011000010110100100000000
Hash               :  0010000100111010
HMAC               :  1001100110010010

PaddedText         :  01001000011000010110100100000000

ENCRYPTION

Round  1  result : 01101001000000001001111011000000
Round  2  result : 10011110110000001000100010000011
Round  3  result : 10001000100000111110100011000101
Round  4  result : 11101000110001011001110101001011
Round  5  result : 10011101010010111000111110010101
Round  6  result : 10001111100101011110011011101001
Round  7  result : 11100110111010011000000100010011
Round  8  result : 10000001000100111011011100100101
Round  9  result : 10110111001001011001011110001001
Round  10  result : 10010111100010010110001111010011
Round  11  result : 01100011110100110111001010100100
Round  12  result : 01110010101001000001110010001010
Round  13  result : 00011100100010100111010111010100
Round  14  result : 011101011101010001011110